# Simple NN

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn')

# Data preparation

In [ ]:
data = pd.read_csv('../../data/SamDysch_glucose_2-5-2022.csv', skiprows=[0])
data.index = pd.to_datetime(data['Device Timestamp'], format="%d-%m-%Y %H:%M")

In [ ]:
# drop non-historic glucose records
data = data[data['Record Type'] == 0]

# only keep bg
to_keep = [
    'Historic Glucose mmol/L',
]
data = data[to_keep]

data = data.rename(columns={'Historic Glucose mmol/L': 'reading'})

data.head()

In [ ]:
# drop NaNs
data = data.dropna()

# Setup hypo threshold

In [ ]:
HYPO_THRESHOLD = 3.9
data['is_hypo'] = (data['reading'] < HYPO_THRESHOLD).astype(int)

In [ ]:
# adding some time variables
data['hour'] = data.index.hour
data['day'] = data.index.dayofweek
data['month'] = data.index.month

# Encode hours

In [ ]:
# OneHotEncode
# data = pd.get_dummies(data, prefix='hour', columns=['hour'])

# sin/cosine encode
data['sin_hour'] = np.sin(2 * np.pi * data['hour'] / 23)
data['cos_hour'] = np.cos(2 * np.pi * data['hour'] / 23)
data = data.drop(['hour'], axis='columns')
print(data.columns)

# creating a lagged and rolling variables
* Was I hypo 15 mins ago? 30 mins ago? Etc
* Rolling average of last N readings
* Sign of gradient of last N readings:
    * I.e., is BG rising, falling, or stable?
    
## Lagged features

In [ ]:
# create lags
# To ensure that we do not make a lag between periods of sensor non-usage, create a new df with the lagged indices & merge onto original data frame
def create_lag(df, lag):
    tolerance = 15 * lag
    freq = '15min'
    print(f'Creating lag of {tolerance} minutes')
    lagged_copy = df[['reading']].shift(lag, freq=freq)
    lagged_copy.rename(columns={'reading': f'lagged_reading_{lag}'}, inplace=True)
    
    merged = pd.merge_asof(df, lagged_copy, left_index=True, right_index=True, direction='backward', tolerance=pd.Timedelta(minutes=tolerance))
    # merged = pd.merge_asof(copy, lagged_copy, left_index=True, right_index=True, direction='backward')
    return merged

NLAGS = 8
for lag in range(1, NLAGS):
    data = create_lag(data, lag)

In [ ]:
# For ease of variable calculation, drop the nans
data = data.dropna()

In [ ]:
# lagged hypo bools
for lag in range(1, NLAGS):
    data[f'is_lagged_hypo_{lag}'] = (data[f'lagged_reading_{lag}'] < HYPO_THRESHOLD).astype(int)

## Rolling features

In [ ]:
# simple differences of lags - was reading higher, lower, or stable?
for lag in range(2, NLAGS):
    data[f'diff_{lag}'] = data['lagged_reading_1'] - data[f'lagged_reading_{lag}']

# gradients - how quick is BG changing?
interval = 15
for lag in range(2, NLAGS):
    data[f'rate_{lag}'] = data[f'diff_{lag}'] / (interval * lag)

## train, test, validation split

In [ ]:
TRAIN_SPLIT = 0.5
VAL_SPLIT = 0.35
TEST_SPLIT = 0.15

In [ ]:
itrain = int(TRAIN_SPLIT * len(data))
ival = int(VAL_SPLIT * len(data))
itest = int(TEST_SPLIT * len(data))

train_data = data.iloc[:itrain]
val_data = data.iloc[itrain:itrain + ival]
test_data = data.iloc[itrain + ival:]

# Variable selection

In [ ]:
rates_and_diffs = [f'diff_{v}' for v in range(2, NLAGS)]
rates_and_diffs.extend([f'rate_{v}' for v in range(2, NLAGS)])

# to fairly compare with baseline, drop any historical variables with time delta < 45 mins
vars_to_drop = [
    'month',
    'day',
    'reading',
    'is_lagged_hypo_1',
    'is_lagged_hypo_2',
    'lagged_reading_1',
    'lagged_reading_2',
    
    'is_lagged_hypo_3',
    'is_lagged_hypo_4',
    'is_lagged_hypo_5',
    'is_lagged_hypo_6',
    'is_lagged_hypo_7',
]
vars_to_drop.extend(rates_and_diffs)

train_data = train_data.drop(vars_to_drop, axis='columns')
val_data = val_data.drop(vars_to_drop, axis='columns')
test_data = test_data.drop(vars_to_drop, axis='columns')

print(train_data.columns)
print(val_data.columns)
print(test_data.columns)

In [ ]:
target = 'is_hypo'

X_train = train_data.drop([target], axis='columns')
y_train = train_data[target]

X_val = val_data.drop(target, axis='columns')
y_val = val_data[target]

X_test = test_data.drop(target, axis='columns')
y_test = test_data[target]

print(X_train.columns)

# class weights

In [ ]:
# Suggestion here is to scale both classes to 50% of total: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
neg = y_train[y_train == 0].count()
pos = y_train[y_train == 1].count()
total = neg + pos

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
print(weight_for_0, weight_for_1)

# model setup

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, TruePositives, FalseNegatives, TrueNegatives, FalsePositives, BinaryAccuracy, AUC

input_shape = (X_train.shape[1],)
activation = 'relu'
learning_rate = 0.001

optimizer = Adam(learning_rate=learning_rate)

# normalize data
norm_layer = Normalization()
norm_layer.adapt(X_train)


model = Sequential()
# preprocessing
model.add(norm_layer)

# FF part
model.add(Dense(40, input_shape=input_shape, activation='relu', kernel_regularizer='l2'))
model.add(Dropout(0.4))
model.add(LayerNormalization())
# model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='relu', kernel_regularizer='l2'))
model.add(Dense(1, activation='sigmoid'))

METRICS = [
      # TruePositives(name='tp'),
      # FalsePositives(name='fp'),
      # TrueNegatives(name='tn'),
      # FalseNegatives(name='fn'), 
      # BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      # AUC(name='auc'),
      AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=METRICS)

model.summary()

# fit model

In [ ]:
# Need a relatively large batch size, to ensure a good chance that positive samples make it into each weight update
# batch_size = 128
batch_size = 64
epochs = 200
callbacks = None
class_weight = {
    0: weight_for_0,
    1: weight_for_1,
}
validation_data = (X_val, y_val)

history = model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    validation_data=validation_data,
    class_weight=class_weight,
    callbacks=callbacks,
    epochs=epochs,
    verbose=1,
    shuffle=True,
)

In [ ]:
# learning curve
fig, ax = plt.subplots(1, 3, figsize=(45, 10))

ax[0].plot(history.history['loss'], label='train')
ax[0].plot(history.history['val_loss'], label='validation')
ax[0].set_ylabel('loss')
ax[0].set_xlabel('epoch')
ax[0].legend(loc='best')

ax[1].plot(history.history['precision'], label='train')
ax[1].plot(history.history['val_precision'], label='validation')
ax[1].set_ylabel('precision')
ax[1].set_xlabel('epoch')
ax[1].legend(loc='best')

ax[2].plot(history.history['recall'], label='train')
ax[2].plot(history.history['val_recall'], label='validation')
ax[2].set_ylabel('recall')
ax[2].set_xlabel('epoch')
ax[2].legend(loc='best')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, fbeta_score
y_pred = model.predict(X_test)
y_pred = y_pred > 0.5

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1: {f1_score(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2)}')

cm = confusion_matrix(y_test, y_pred, normalize='all')
sns.heatmap(cm, annot=True, square=True)